In [1]:
### Item-based collaborative filtering рекомендательная система
##### HR@10  0.092

# Games RSs

In [2]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Данные взяты отсюда - http://jmcauley.ucsd.edu/data/amazon/
# http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
JSON_DATA_PATH = "./reviews_Video_Games_5.json"
N = 10

## Загрузка данных

In [4]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
    return pd.DataFrame(rows, columns=cols)

In [5]:
%%time
df = get_data_frame()

CPU times: user 2.18 s, sys: 112 ms, total: 2.29 s
Wall time: 2.31 s


## Готовим выборки

In [6]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [7]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


 ## Метрика

In [8]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [9]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

## ItemBased Top-K Recommender

In [10]:
from tqdm import tqdm

class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}

In [11]:
#########################################################################
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
def load_data(df):
    rows = []
    cols = []
    data = []
    
    uid_to_row = {}
    iid_to_col = {}
    
    mean_ratings=df.groupby(df.uid).rating.mean()
    
    for t in df.itertuples():
        row_id = uid_to_row.setdefault(t.uid, len(uid_to_row))
        col_id = iid_to_col.setdefault(t.iid, len(iid_to_col))
        rating = t.rating
        
        rows.append(row_id)
        cols.append(col_id)
        data.append(1.0) #игнорируем рейтинги
        
    ui_m = csr_matrix((data, (rows, cols)))
    return ui_m, uid_to_row, iid_to_col
#########################################################################
# вспомогательные функции, которые могут пригодиться при построении Item-based CF
def nullify_main_diagonal(m):
    positions = range(m.shape[0])
    eye = csr_matrix((np.ones(len(positions)), (positions, positions)), m.shape)
    return m - m.multiply(eye)
def get_topk(matrix, top, axis=1):
    """Converts source matrix to Top-K matrix
    where each row or column contains only top K values

    :param matrix: source matrix
    :param top: number of top items to be stored
    :param axis: 0 - top by column, 1 - top by row
    :return:
    """
    rows = []
    cols = []
    data = []

    if axis == 0:
        matrix = matrix.T.tocsr()

    for row_id, row in enumerate(matrix):
        if top is not None and row.nnz > top:
            top_args = np.argsort(row.data)[-top:]

            rows += [row_id] * top
            cols += row.indices[top_args].tolist()
            data += row.data[top_args].tolist()
        elif row.nnz > 0:
            rows += [row_id] * row.nnz
            cols += row.indices.tolist()
            data += row.data.tolist()

    topk_m = csr_matrix((data, (rows, cols)), (matrix.shape[0], matrix.shape[1]))

    if axis == 0:
        topk_m = topk_m.T.tocsr()

    return topk_m
######################################################################### 

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import time

class ItemBasedTopKRecommender(BasicRecommender):
    def __init__(self,TOPK=30,verbose=0):
        super(BasicRecommender, self).__init__()
        self.TOPK=TOPK
        self.verbose=verbose
    def fit(self,training_df):

        self.log("fit on %d records"%(len(training_df)))
        start=time.time()
        self.ui_m, self.uid_to_row, self.iid_to_col = load_data(training_df)
        self.col_to_iid = {col_id: iid for iid, col_id in self.iid_to_col.items()}
        self.log("load_data(): %0.1f sec"%(time.time()-start))
        
        
        #Построение sim-матрицы
        start=time.time()
        self.ii_sim_m = cosine_similarity(self.ui_m.T.tocsr(), dense_output=False)
        self.ii_sim_m = nullify_main_diagonal(self.ii_sim_m)
        #оставляем в строках и столбцах только Top-K компонент
        self.ii_sim_m = get_topk(self.ii_sim_m, top=self.TOPK,axis=1)
        self.ii_sim_m = get_topk(self.ii_sim_m, top=self.TOPK,axis=0)                
        #нормализуем матрицу
        self.ii_sim_m_norm=csr_matrix(normalize(self.ii_sim_m,axis=1))
        self.log("build sim matrix: %0.1f sec"%(time.time()-start))
             
        #Создаем словарик user-item чтобы в дальнейшем не рекомендовать уже купленные игры
        self.used_items=dict()
        for t in training_df.itertuples():
            self.used_items[(t.uid,t.iid)]=True
    
    def get_recs(self, uid, top=N):
        recs = []
        if uid in self.uid_to_row:
            u_row_id = self.uid_to_row[uid]
            u_row = self.ui_m[u_row_id]

            predicted_prob=self.ii_sim_m_norm.dot(u_row.T).T.tocsr()  #/u_row.sum()
            top_args = np.argsort(-predicted_prob.data)

            for arg_id in top_args:
                if len(recs)>=top:
                    #если уже набралось N рекомендаций, на выход!
                    break
                col_id = predicted_prob.indices[arg_id]
                iid=self.col_to_iid[col_id]
                if not ((uid,iid) in self.used_items):
                    #пропускаем уже купленные мгры
                    score = predicted_prob.data[arg_id]
                    recs.append((iid,score))
        return recs
    def log(self,msg):
        if self.verbose>0:
            print(msg,flush=True)
    
    def get_batch_recs(self, uids, top=N):
        rez=dict()
        self.log("Get recommendations: ")        
        it=uids
        if self.verbose:
            it=tqdm(it)
        
        for uid in it:
            rez[uid]=dict(self.get_recs(uid,top)[:top])
        return rez

## Финальный тест

In [13]:
%%time
recommender=ItemBasedTopKRecommender(verbose=True)
recommender.fit(training_df)
recs_dict=recommender.get_batch_recs(test_df.uid.unique())
value=hit_ratio(recs_dict,test_dict)   
print("HR: ",value)

fit on 185427 records
load_data(): 0.3 sec
build sim matrix: 1.5 sec
Get recommendations: 


100%|██████████| 6815/6815 [00:06<00:00, 1124.96it/s]

HR:  0.09200293470286133
CPU times: user 8.06 s, sys: 32 ms, total: 8.1 s
Wall time: 8.09 s


In [14]:
print("HR: ",value)

HR:  0.09200293470286133
